In [1]:
import pandas as pd

In [2]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')
bike_realtime_df.drop(columns="create_time",inplace=True)

In [3]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["month"] = bike_realtime_df["source_time"].dt.month
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [4]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [5]:
bike_realtime_df = bike_realtime_df.loc[(bike_realtime_df["month"]==4)&((bike_realtime_df["hour"]>5) |(bike_realtime_df["hour"]==0)),].reset_index(drop=True)

In [6]:
mrt_history_df["month"] = mrt_history_df["date"].dt.month
mrt_history_df = mrt_history_df.loc[(mrt_history_df["month"]==4)&((mrt_history_df["hour"]>5) |(mrt_history_df["hour"]==0)),].reset_index(drop=True)

In [7]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [8]:
df  = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","total_space","lat","lng"]],
                              how="left",
                              on="bike_station_id")

In [9]:
df = df.merge(time_table_df.loc[:,["date","day_of_week","day_of_week_name"]],
            on="date",
            how="left")


In [10]:
idx = youbike_mrt_distance_df.groupby("bike_station_id")["distance"].idxmin()
youbike_min_distance_table = youbike_mrt_distance_df.loc[idx,].reset_index(drop=True)

In [11]:
mrt_station_table = youbike_min_distance_table.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500101001,BR08,0.015941,科技大樓
1,500101002,BR08,0.053192,科技大樓
2,500101003,BR08,0.302305,科技大樓
3,500101004,BR08,0.296290,科技大樓
4,500101005,BR08,0.510676,科技大樓
...,...,...,...,...
1410,500119087,G07,0.645005,公館
1411,500119088,BR08,0.683471,科技大樓
1412,500119089,G07,0.884494,公館
1413,500119090,G07,0.719453,公館


In [12]:
df = df.merge(mrt_station_table,on="bike_station_id",how="left")
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,distance,station_name
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360,3,Tuesday,BR08,0.015941,科技大樓
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
3,500101001,2024-04-17,6,23.500000,4.500000,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
4,500101001,2024-04-17,7,17.833333,10.166667,28,25.02605,121.54360,4,Wednesday,BR08,0.015941,科技大樓
...,...,...,...,...,...,...,...,...,...,...,...,...,...
346896,500119091,2024-04-29,16,11.500000,6.500000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
346897,500119091,2024-04-29,17,9.000000,9.000000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
346898,500119091,2024-04-29,18,0.500000,17.500000,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓
346899,500119091,2024-04-29,19,6.666667,11.333333,18,25.01816,121.54469,2,Monday,BR08,0.890423,科技大樓


In [13]:
df.dtypes

bike_station_id              int64
date                datetime64[ns]
hour                         int32
aval_bike_mean             float64
aval_space_mean            float64
total_space                  int64
lat                        float64
lng                        float64
day_of_week                  int64
day_of_week_name            object
mrt_station_id              object
distance                   float64
station_name                object
dtype: object

In [14]:
mrt_history_df

,date,hour,mrt_station,enter_num,exit_num,month
0,2024-04-01,0,龍山寺,144,153.0,4
1,2024-04-01,0,麟光,15,21.0,4
2,2024-04-01,0,頭前庄,10,35.0,4
3,2024-04-01,0,頂溪,85,219.0,4
4,2024-04-01,0,頂埔,13,100.0,4
...,...,...,...,...,...,...
67255,2024-04-30,23,三重國小,116,423.0,4
67256,2024-04-30,23,三重,87,183.0,4
67257,2024-04-30,23,三民高中,89,453.0,4
67258,2024-04-30,23,三和國中,99,356.0,4


In [16]:
df = df.merge(mrt_history_df,left_on=["station_name","date","hour"],right_on=["mrt_station","date","hour"],how="left")

In [19]:
df["bike_rate"] = df["aval_bike_mean"]/df["total_space"]

In [20]:
df.to_csv("test2.csv",encoding="utf-8-sig",index=False)

In [21]:
#############################